In [1]:
%cd ..

/Users/philipphager/Developer/ultr-cm-vs-ips


In [2]:
from util import load_experiment

/usr/local/Caskroom/miniconda/base/envs/ultr-cm-vs-ips/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
baseline_df, val_df, test_df = load_experiment("dataset_size")

In [3]:
def plot(dataset_df, legend=True, width=600, height=200, metric="average_relevant_position", title="", y=[0, 1.0]): 
    lines = alt.Chart(dataset_df, width=width, height=height, title=title).mark_line().encode(
        x=alt.X("n_sessions", scale=alt.Scale(type="log"), title="Number of Train Queries", axis=alt.Axis(format="e")),
        y=alt.Y(f"mean({metric})", scale=alt.Scale(zero=False, domain=y)),
        color=alt.Color("model", legend=None),
    )

    marks = alt.Chart(dataset_df).mark_point().encode(
        x=alt.X("n_sessions", scale=alt.Scale(type="log"), title="Number of Train Queries", axis=alt.Axis(format="e")),
        y=alt.Y(f"mean({metric})", scale=alt.Scale(zero=False)),
        shape=alt.Shape("model", legend=alt.Legend(orient="bottom"), title="") if legend else alt.Shape("model", legend=None),
        color=alt.Color("model", legend=alt.Legend(orient="bottom"), title="") if legend else alt.Color("model", legend=None),
    )

    ci = alt.Chart(dataset_df).mark_errorband(opacity=0.5).encode(
        x=alt.X("n_sessions", scale=alt.Scale(type="log"), title="Number of Train Queries", axis=alt.Axis(format="e")),
        y=alt.Y(metric, scale=alt.Scale(zero=False)),
        color=alt.Color("model", legend=None),
    )

    return alt.layer(
        lines,
        marks,
        ci
    ).resolve_scale(
        color="independent",
        shape="independent",
    )

yahoo_df = test_df[test_df.dataset == "Yahoo"]
mslr_df = test_df[test_df.dataset == "MSLR-Web30K"]

(
    plot(mslr_df, legend=True, metric="nDCG@10", title="MSLR-WEB30K", y=[0.25, 0.45]) &
    plot(yahoo_df, legend=True, metric="nDCG@10", title="Yahoo", y=[0.59, 0.74])
)

/usr/local/Caskroom/miniconda/base/envs/ultr-cm-vs-ips/lib/python3.9/site-packages/altair/utils/core.py:185: UserWarning: I don't know how to infer vegalite type from 'empty'.  Defaulting to nominal.
  warnings.warn(


alt.VConcatChart(...)

In [4]:
yahoo_df[yahoo_df.n_sessions == 100_000_000].groupby("model").epoch.describe()

,count,mean,std,min,25%,50%,75%,max
model,,,,,,,,
Neural PBM - Unbiased,1.0,49.0,NaN,49.0,49.0,49.0,49.0,49.0
Pointwise IPS - Unbiased,1.0,37.0,NaN,37.0,37.0,37.0,37.0,37.0


In [5]:
yahoo_df[yahoo_df.n_sessions == 100_000_000].groupby("model").nDCG.describe()

,count,mean,std,min,25%,50%,75%,max
model,,,,,,,,
Neural PBM - Unbiased,1.0,0.774008,NaN,0.774008,0.774008,0.774008,0.774008,0.774008
Pointwise IPS - Unbiased,1.0,0.780237,NaN,0.780237,0.780237,0.780237,0.780237,0.780237
